# Typical Start

`nbdev` needs to know where to export the module code fomr this notebook. You will typically just export to a module name the same as the notebook but you can rename and add extra directories to create a package structure if you wish

In [ ]:
# hide
# default_exp hello_sciflow

The below code is an example of how to define a step. A step is a function we chain together to form a workflow. Typically they will have inputs and return values which can be used in subsequent steps but we start simple here.

In [ ]:
# export

from pathlib import Path

from sciflow.run_flow import check_call_flow, run_flow_async
from sciflow.to_metaflow import sciflow_metaflow

## Breaking down the named step directive:

* `exportn_step:first` = export a named step called **first**

In [ ]:
# exportn_step:first


def first():
    print("The first step")

Auto-reloading modules is very useful when using `nbdev` as changes to underlying modules are picked up without having to restart the kernel.

In [ ]:
%load_ext autoreload
%autoreload 2

You need to export all code that is gonig to be used in flows - this means things like import statements as well as function definitions.

In [ ]:
# export

# Params

* `sciflow` uses the papermill format for paramaeterising notebooks. 
* See here for how to specify papermill params: https://papermill.readthedocs.io/en/latest/usage-parameterize.html. 
* These parameters will be available to use or edit in your flows.

In [ ]:
# export

traffic_percent = 1

In [ ]:
# exportn_step:second


def second(traffic_percent):
    results = {}
    print(f"Traffic percent: {traffic_percent}")
    results["traffic_multiplied"] = traffic_percent * 2
    return results

# See the results

## 1. Command Line Interface

# 1.1  Convert the notebooks to sciflow modules

In [ ]:
# tst
!sciflow_build_lib

# 1.2 Tests that the notebooks work!

In [ ]:
# tst
!nbdev_test_nbs --pause=2

# 1.3 See the created module

In [ ]:
# tst
!cat sciflow_examples/hello_sciflow.py

# Create a `Flow`

In [ ]:
!sciflow_metaflow --track False

# Run this notebook as a `Flow` (this example uses `metaflow`)

In [ ]:
!source ~/.sciflow/env && python flows/metaflow/hello_sciflow.py

In [ ]:
# tst
!source ~/.sciflow/env && python flows/metaflow/hello_sciflow.py --no-pylint run

# 2. Programmatically

In [ ]:
# tst
print(
    check_call_flow(Path("flows", "metaflow", "hello_sciflow.py"), flow_command="run")[
        1
    ]
)

# You really are editing your current workflow - so you can keep modifying it as you go..

In [ ]:
# exportn_step:third


def third(traffic_percent):
    print(f"Traffic * 3={traffic_percent * 3}")

.. then convert it

In [ ]:
# tst
!sciflow_build_lib

In [ ]:
sciflow_metaflow(False)

## You can call this workflow using different parameters.

In [ ]:
# tst
print(
    check_call_flow(
        Path("flows", "metaflow", "hello_sciflow.py"),
        flow_command="run",
        params={"traffic_percent": 50},
    )[1]
)

## Async Flow Running

You can also run flows in the background on your local machine or remotely using `run_flow_async`. This is useful if you are writing a notebook, have an idea to try something but want to keep thinking and exploring the problem space while you test out that idea.

In [ ]:
# slow

task = run_flow_async(
    Path(Path(".").resolve(), "02_hello_sciflow.ipynb"),
    params={"traffic_percent": 100},
)
task

### Do something else here...

In [ ]:
# slow
await task
assert 0 == task.result()[0]

In [ ]:
print(task.result()[1])